In [1]:
import json

from kafka import KafkaConsumer

SyntaxError: invalid syntax (simple.py, line 54)

In [2]:
#https://www.geeksforgeeks.org/how-to-install-and-run-apache-kafka-on-windows/

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [ ]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='John',
    last_name='Doe'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

Create a consumer without subscribing to any particular topic

In [ ]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [ ]:
general_consumer.topics()

Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
general_consumer.close()

In [ ]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [ ]:
consumer.subscription()

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
consumer.close()